# Implementing a custom LLM-EC algorithm

In this tutorial we implement a simple optimizer and compare it with **LLaMEA** and **EoH**.

In [ ]:
from iohblade.method import Method
from iohblade import Experiment, Ollama_LLM
from iohblade.methods import LLaMEA, EoH
from iohblade.problems import BBOB
from iohblade.loggers import ExperimentLogger

In [ ]:
class MyLLMOptimizer(Method):
    def __init__(self, llm, budget=20, name='MyLLM'):
        super().__init__(llm, budget, name)

    def __call__(self, problem):
        msg = [{'role':'user','content': problem.get_prompt()}]
        best = problem(self.llm.sample_solution(msg))
        for _ in range(self.budget - 1):
            msg = [
                {'role':'user','content': problem.get_prompt()},
                {'role':'assistant','content': best.code},
                {'role':'user','content': 'Improve the algorithm.'}
            ]
            cand = problem(self.llm.sample_solution(msg))
            if cand.fitness > best.fitness:
                best = cand
        return best

    def to_dict(self):
        return {'method_name': self.name, 'budget': self.budget}

In [ ]:
llm = Ollama_LLM('qwen2.5-coder:14b')
budget = 30
methods = [
    LLaMEA(llm, budget=budget, name='LLaMEA'),
    EoH(llm, budget=budget, name='EoH'),
    MyLLMOptimizer(llm, budget=budget, name='MyLLM'),
]
problems = [BBOB(functions=[1], instances=[1], dims=[5], budget_factor=200, name='BBOB')]
logger = ExperimentLogger('results/custom_method')
experiment = Experiment(methods=methods, problems=problems, llm=llm, runs=3, show_stdout=True, exp_logger=logger)

In [ ]:
experiment()